In [ ]:
import cv2
import mediapipe as mp
import time

# Mediapipe setup
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.75)

# Traffic light states
current_state = "RED"

# Map gesture count to traffic light state
gesture_to_signal = {
    1: "GREEN",
    2: "YELLOW",
    5: "RED"
}

def count_fingers(hand_landmarks):
    tips_ids = [4, 8, 12, 16, 20]
    fingers = []

    # Thumb
    fingers.append(hand_landmarks.landmark[tips_ids[0]].x < hand_landmarks.landmark[tips_ids[0] - 1].x)

    # Fingers
    for id in range(1, 5):
        fingers.append(
            hand_landmarks.landmark[tips_ids[id]].y < hand_landmarks.landmark[tips_ids[id] - 2].y
        )

    return fingers.count(True)

def draw_traffic_light(frame, state):
    colors = {"RED": (0, 0, 255), "YELLOW": (0, 255, 255), "GREEN": (0, 255, 0)}
    cv2.rectangle(frame, (450, 50), (500, 200), (50, 50, 50), -1)
    pos_map = {"RED": 70, "YELLOW": 120, "GREEN": 170}
    for state_name, pos in pos_map.items():
        color = colors[state_name] if state == state_name else (100, 100, 100)
        cv2.circle(frame, (475, pos), 15, color, -1)
    return frame

# Start camera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
        

    frame = cv2.flip(frame, 1)
    h, w, c = frame.shape

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    if result.multi_hand_landmarks:
        for handLms in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, handLms, mp_hands.HAND_CONNECTIONS)
            count = count_fingers(handLms)
            if count in gesture_to_signal:
                current_state = gesture_to_signal[count]

    frame = draw_traffic_light(frame, current_state)
    cv2.putText(frame, f"Signal: {current_state}", (10, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    cv2.imshow("AI Traffic Controller", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()